<a href="https://colab.research.google.com/github/jonny243/github-slideshow/blob/main/jc_individual_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
import pandas as pd
import pickle
!pip install geopandas
import geopandas as gpd
from shapely.geometry import Point


drive.mount('/content/drive')
!ls "/content/drive/My Drive"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 5.1 MB/s 
     |████████████████████████████████| 6.3 MB 34.9 MB/s 
     |████████████████████████████████| 16.7 MB 32.4 MB/s 
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'220523 Interim report .docx'  'Getting started.pdf'
'Colab Notebooks'	        Imperial
 data			        intro2ml_cw1_07-master-data.zip


In [17]:
trans_file = "/content/drive/My Drive/Colab Notebooks/data/transaction_data/pp-complete-CHANGED.csv"
df = pd.read_csv(trans_file)
df.rename(columns={"Date_of_transfer": "date", "Postcode" : "postcode"}, inplace=True)
print(df)

                  Transaction_unique_identifier   Price              date  \
0        {680A511A-C9CD-4E26-84EC-B6697A53CEF8}  122000  29/09/1995 00:00   
1        {FC3888F4-0BE9-452F-A2EA-B66982729FAF}   37000  22/12/1995 00:00   
2        {024F306B-9F59-4D5F-8EE9-B6698ED0A538}   93500  27/07/1995 00:00   
3        {4326AC9E-5FD9-47D1-AA48-B66993995051}   32000  03/03/1995 00:00   
4        {B6BC448E-2915-4C44-8AE5-AF5978A922E4}   38000  08/09/1995 00:00   
...                                         ...     ...               ...   
1048569  {2EC63321-82C0-4155-BB2E-5CEAA0B2D5E2}  111000  25/11/1996 00:00   
1048570  {54CD17AA-2048-48CB-B75F-5CEABA8E10CF}  128750  23/10/1996 00:00   
1048571  {7705516E-D617-4B9C-BB40-592A7827B5B4}  122000  19/07/1996 00:00   
1048572  {98825484-BDED-4EB2-8E6F-592A803F170F}  120000  04/10/1996 00:00   
1048573  {7B929AED-B027-4F5A-BAE4-592A8163FD4E}  177000  04/09/1996 00:00   

         postcode Property_type Old_new Freehold_leasehold         POAN  \


In [13]:
# add whether in flood zone 
def add_flood_zone_data_to_trans_df(trans_df):
    # import historic flooding events as a geo dataframe 
    flood_file = "/content/drive/My Drive/Colab Notebooks/data/Recorded_Flood_Outlines/Recorded_Flood_Outlines.shp"
    # flood_file = project_folder/"data/Recorded_Flood_Outlines/Recorded_Flood_Outlines.shp"
    gdf = gpd.read_file(flood_file)
    gdf = gdf.sort_values(by='end_date', ascending=False)
    # remove flooding events that are greater than 2025 (note dates are either correct or 2050)
    gdf['end_date'] = pd.to_datetime(gdf['end_date'])
    gdf['start_date'] = pd.to_datetime(gdf['start_date'])
    gdf = gdf[(gdf['end_date'] < '2025-04-01')]
    postcode_df = get_easting_northing_df()
    trans_df['date'] = pd.to_datetime(trans_df['date'])
    print('Starting merging')
    # add northing, easting, long and lat to transaction df
    trans_df_app = trans_df.merge(postcode_df, how = 'inner', left_on = 'postcode', right_on = 'pcd')
    # trans_df_app = add_postcode_agg_data(trans_df_app)
    print("Finish merging")
    trans_df_app = trans_df_app.join(
        trans_df_app.apply(
            lambda row: last_flood(gdf, row.oseast1m, row.osnrth1m, row.date), axis="columns"
        ))
    print("Added flooding information")
    print(trans_df_app)
    # for each postcode, create a new column with whether flooded or not 
    
    # trans_df = trans_df.join(df.apply(lambda row: last_flooding_event_bng(postcode_df, gdf, row.postcode, int(row.date[0:4])), axis="columns"))
    # df['last_year'] = df.apply(lambda row: last_flooding_event_bng(postcode_df, gdf, row.postcode, int(row.date[0:4])), axis=1)
    
    # print(df)
    trans_df_app.to_pickle(project_folder/"data/transaction_data/pp-mini-pkl-updated-years-postcode-csv.pkl")
  
    

In [12]:
def add_postcode_agg_data(df):    
    econ_activity_file = "/content/drive/My Drive/Colab Notebooks/data/postcode_aggregate_data/economic_activity.csv"
    health_file = "/content/drive/My Drive/Colab Notebooks/data/postcode_aggregate_data/health.csv"
    highest_qual_file = "/content/drive/My Drive/Colab Notebooks/data/postcode_aggregate_data/highest_qualification.csv"
    people_per_room_file = "/content/drive/My Drive/Colab Notebooks/data/postcode_aggregate_data/people_per_room.csv"
    pop_density_file = "/content/drive/My Drive/Colab Notebooks/data/postcode_aggregate_data/population_density.csv"
    rooms_central_heating_file = "/content/drive/My Drive/Colab Notebooks/data/postcode_aggregate_data/rooms_central_heating.csv"
    df1 = pd.read_csv(econ_activity_file)
    df2 = pd.read_csv(health_file)
    df3 = pd.read_csv(highest_qual_file)
    df4 = pd.read_csv(people_per_room_file)
    df5 = pd.read_csv(pop_density_file)
    df6 = pd.read_csv(rooms_central_heating_file)
    
    print(df1)
    print(df2)
    print(df3)
    print(df4)
    print(df5)
    print(df6)

    postcode_agg = df1.merge(df2, how = 'inner', left_on = 'postcode sector', right_on = 'postcode sector')
    postcode_agg = postcode_agg.merge(df3, how = 'inner', left_on = 'postcode sector', right_on = 'postcode sector')
    postcode_agg = postcode_agg.merge(df4, how = 'inner', left_on = 'postcode sector', right_on = 'postcode sector')
    postcode_agg = postcode_agg.merge(df5, how = 'inner', left_on = 'postcode sector', right_on = 'postcode sector')
    postcode_agg = postcode_agg.merge(df6, how = 'inner', left_on = 'postcode sector', right_on = 'postcode sector')

    print(postcode_agg)


In [9]:
def get_easting_northing_df():
    postcode_file = "/content/drive/My Drive/Colab Notebooks/data/postcode_data/ONSPD_MAY_2022_UK.csv"
    
    # COLUMN NAMES IN FILE:
    # pcd	pcd2	pcds	dointr	doterm	oscty	ced	oslaua osward	parish	usertype	oseast1m	osnrth1m	osgrdind	
    # oshlthau	nhser	ctry	rgn	streg	pcon	eer	teclec	ttwa pct	itl	statsward	oa01	casward	park	lsoa01	msoa01	ur01ind	oac01	
    # oa11	lsoa11	msoa11	wz11	ccg	bua11	buasd11	ru11ind	oac11	lat	long lep1	lep2	pfa	imd	calncv	stp
    
    # WANTED COLUMNS
    col_list = ['pcd', 'doterm', 'oseast1m','osnrth1m', 'ctry', 'lat', 'long']
    postcode_df = pd.read_csv(postcode_file, usecols=col_list)
    # postcode_df = postcode_df[postcode_df_2['doterm'].isna()]
    postcode_df.drop(['doterm'], axis=1, inplace=True)
    return postcode_df


In [11]:

def last_flood(gdf, easting, northing, date):
    p1 = Point(northing, easting)
    gdf_matches = gdf[gdf.contains(p1)]
    gdf_matches_2 = gdf_matches[gdf_matches['end_date'] < date] 
    if(gdf_matches_2.empty):
        return pd.Series(dict(flood_zone = False, last_flood_year = -1))
    print('Another')
    return pd.Series(dict(flood_zone = True, last_flood_year = gdf_matches_2.iloc[0].end_date))

In [19]:
df = df.head(10000)
add_flood_zone_data_to_trans_df(df)

Starting merging
Finish merging
Added flooding information
             Transaction_unique_identifier   Price       date postcode  \
0   {024F306B-9F59-4D5F-8EE9-B6698ED0A538}   93500 1995-07-27  UB8 2AP   
1   {4326AC9E-5FD9-47D1-AA48-B66993995051}   32000 1995-03-03  HD6 4AD   
2   {B6BC448E-2915-4C44-8AE5-AF5978A922E4}   38000 1995-08-09  B30 3PP   
3   {7020D11D-FCCE-41F7-A266-B69DE4DF82C6}   58000 1995-11-04  HP3 9LS   
4   {DF2A5520-3FFF-4970-BE62-B2E652EAF0DB}  139950 1995-09-15  HA5 2TP   
5   {6B8605E3-41CA-40A0-B2EC-B2E65A3D28B0}   14000 1995-03-24  NG2 4AR   
6   {32091EBB-397A-43EC-AA77-B2E66E75E537}   49995 1995-04-21  CO5 0DD   
7   {58E7F890-9A51-4934-A4A3-B2E67623FF4F}    8000 1995-12-22  BB2 1HT   
8   {9145AE71-E08F-4288-BA10-B669B580065A}   77500 1995-11-17  B60 2HA   
9   {BA0A22AD-5095-4A2F-BCEB-B69E02B7A8EE}   36500 1995-01-12  HU7 0DW   
10  {5F603E31-C552-4D06-9ADB-B69E17807B1D}  105000 1995-03-10  SS8 7NY   
11  {D94CFABA-FA67-4D1B-ADBF-B69E28681B5D}   56500 19

NameError: ignored